In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print('length of the dataset in characters:',len(text))

length of the text: 1115394


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [10]:
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for i,c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hello world!'))
print(decode(encode('hello nanoGPT!')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
hello nanoGPT!


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [17]:
def split_data(percent):
    n = int(percent * len(data))
    train_data = data[:n]
    val_data = data[n:]
    return train_data, val_data

train_data, val_data = split_data(0.9)

In [21]:
torch.manual_seed(13337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x,y

xb, yb = get_batch('train')
xb.shape, yb.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        self.token_emmbding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        logits = self.token_emmbding_table(idx) # (B, T, C)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss 
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T)
        for _ in range(max_new_tokens):

            logits, loss = self(idx)

            logits = logits[:, -1, :] # (B, C)

            probs = F.softmax(logits, dim=-1) # (B, C)

            idx_next = torch.multinomial(probs, num_samples=1)

            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)

        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65]) tensor(4.7864, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [33]:
#create a Pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [36]:
batch_size=32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.49489426612854


In [38]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


By:
NRY:

D t nt s t oud.

LABe h.
Haso tll'd se. a thm ad IV:
IUSE g temy'lo yais d wis, htithance?
HANGes, ck inous analis e
werdors s y Pap
BIOHe arencharre d bes t an d tl:
AMENR sowess? d cl hor, ushene wsthroreforecend, steto inethinlllo!
DENo malve A:
Hoourk theafand o fil'lavis yreeblalis oweay topa bentid her.
Dor, so thous,
W:
Tong sthedethernthakis y ksthy iststhind t te n pao; wif Jur hecoreror t co thyomyer t,
ANThes.
fren lle be pat y INEverndarerfis helousthe atim.

I asshe u ngan
